In [64]:
import pandas as pd
from scipy.stats import pearsonr, chi2_contingency, spearmanr

In [65]:
# đọc dữ liệu
df = pd.read_csv('D:\Desktop\Data_Analyst\Jupiter_Notebook_Pandas\Pandas\house_price_Dống-Da_Hà-Nội_subdata.csv')
df.head()

,area,price,postDate,land_certificate,house_direction,balcony_direction,toilet,bedroom,floor,lat,long,property_type
0,50.0,14700.0,44197,So do,NaN,NaN,4.0,6.0,6.0,21.015698,105.824583,trong ngo
1,57.0,4200.0,44265,So do,Nam,Nam,3.0,8.0,4.0,21.025208,105.833111,trong ngo
2,62.0,11000.0,44268,So do,NaN,dong-Nam,6.0,5.0,6.0,21.025379,105.801011,trong ngo
3,26.0,2900.0,44292,So do,NaN,NaN,4.0,3.0,4.0,21.014325,105.834918,trong ngo
4,40.0,3950.0,44292,So do,NaN,NaN,NaN,NaN,5.0,21.009862,105.838083,trong ngo


In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   area               665 non-null    float64
 1   price              617 non-null    float64
 2   postDate           671 non-null    int64  
 3   land_certificate   369 non-null    object 
 4   house_direction    42 non-null     object 
 5   balcony_direction  24 non-null     object 
 6   toilet             240 non-null    float64
 7   bedroom            320 non-null    float64
 8   floor              366 non-null    float64
 9   lat                671 non-null    float64
 10  long               671 non-null    float64
 11  property_type      671 non-null    object 
dtypes: float64(7), int64(1), object(4)
memory usage: 63.0+ KB


#### Giữa giá nhà và diện tích có tương quan với nhau

In [67]:
# H0:r=0 giữa hai thuộc tính có không có mối tương quan
# H1:r≠0 giữ hai thuộc tính có mối tương quan

In [68]:
df1 = df[['area', 'price']]
df1.dropna(inplace=True)
r, p = pearsonr(df1['area'], df1['price'])
print('r=%.3f, p=%.3f' % (r, p))

r=0.229, p=0.000


C:\Users\Admin\AppData\Local\Temp/ipykernel_21264/630210109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.dropna(inplace=True)


In [69]:
# Kết luận pvalue=0.00 <0.05 nên hai thuộc tính diện tích và giá tương quan với nhau
# Vì 0.2<=|r|<0.4: => giữa hai thuộc tính có tương quan tuyến tính yếu

#### Giữa giá nhà và tọa độ địa lý (lat, long) có tương quan với nhau

In [70]:
df2 = df[['area', 'lat', 'long']]
df2.dropna(inplace=True)
r1, p1 = pearsonr(df2['area'], df2['lat'])
r2, p2 = pearsonr(df2['area'], df2['long'])
print('r1=%.3f, p1=%.3f' % (r1, p1))
print('r2=%.3f, p2=%.3f' % (r2, p2))

r1=-0.054, p1=0.167
r2=0.068, p2=0.079


C:\Users\Admin\AppData\Local\Temp/ipykernel_21264/4292479415.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)


In [71]:
# Vì pvalue > 0.05 nên giữa thuộc tính không có tính quan với nhau

#### Giữa thuộc tính land_certificate và property_type có tương quan với nhau

In [72]:
df3 = df[['land_certificate', 'property_type']]
df3.fillna(value="Không có",inplace=True)
contingency = pd.crosstab(df3['land_certificate'], df3['property_type'])
# print(contingency)
c, p, dof, expected = chi2_contingency(contingency)
print(p)

0.21106331497658834


d:\Python\lib\site-packages\pandas\core\frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [73]:
# Kết luận: Do p > 0.05 , chưa đủ cơ sở để kết luận giữa hai thuộc tính này có mối tương quan với nhau

#### Hãy  mã hóa lại thuộc tính giá nhà theo đơn vị (nghìn đồng/m2) và sắp xếp giá nhà thành 4 mức tương ứng với các khoảng tứ phân vị. Tiến hành kiểm định tương quan giữa biến giá nhà này với các biến tọa độ vị trí (lat, long)

In [74]:
df4 = df.filter(['area', 'price', 'lat', 'long'])
df4.dropna(inplace=True)
df4['price/m2'] = df4.area * df4.price
q1, q2, q3 = df4['price/m2'].quantile(0.25), df4['price/m2'].quantile(0.5), df4['price/m2'].quantile(0.75)

In [75]:
# tiến hành tính các khoảng tứ phân vị của price/m2
def price_order(price):
    if price < q1:
        return 1
    elif price >= q1 and price < q2:
        return 2
    elif price >= q2 and price < q3:
        return 3
    else:
        return 4        

In [76]:
# tiến hành mã hoá
df4['price_order'] = df4['price/m2'].apply(price_order)

In [79]:
r1, p1 = spearmanr(df4.lat, df4.price_order)
r2, p2 = spearmanr(df4.long, df4.price_order)
print ("r1= ", r1, "; p1=: ", p1)
print ("r2= ", r2, "; p2=: ", p2)

r1=  -0.05015432208161317 ; p1=:  0.21422250371067147
r2=  -0.20556501754775228 ; p2=:  2.711141259984569e-07


In [ ]:
# Kết luận Vì p > 0.05 chưa đủ cơ sở để kết luận giữa hai thuộc tính này có mối liên hệ với nhau